# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [15]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [46]:
len(data)

4870

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


## 1. What test is appropriate for this problem? Does CLT apply?

* <b>2 proportion z-test</b> is appropriate for this problem to compare the two proportions (call back rate) of interest
* Check if CLT applies below:

In [26]:
b = data[data.race == 'b'].call
w = data[data.race == 'w'].call
print('b group calls and no calls:')
b.value_counts()

b group calls and no calls:


0.0    2278
1.0     157
Name: call, dtype: int64

In [25]:
print('w group calls and no calls:')
w.value_counts()

w group calls and no calls:


0.0    2200
1.0     235
Name: call, dtype: int64

Since numbers of successes and failures for both groups are greater than 10, data can be approximately modeled as normal and <b>CLT applies</b>.

## 2. What are the null and alternate hypotheses?

* Null hypothesis - call back rate <b>is the same</b> for black-sounding names and white-sounding names
* Alternate hypothesis - call back rate <b>is different</b> for black-sounding names and white-sounding names

## 3. Compute margin of error, confidence interval, and p-value.

In [40]:
N_b = len(b)
N_w = len(w)
p_b = b.mean()
p_w = w.mean()

p_diff = abs(p_b - p_w)
SE = math.sqrt(p_b * (1 - p_b) / N_b + p_w * (1 - p_w) / N_w)
CI = stats.norm.interval(0.95, loc=p_diff, scale=SE)

z = p_diff / SE
p_value = stats.norm.sf(abs(z))*2

In [41]:
print('Confidence Interval (95%): ' + str(CI))
print('Margin of Error: ' + str(abs(CI[0] - CI[1])/2/p_b*100) + '%')
print('p-value: ' + str(p_value))

Confidence Interval (95%): (0.016777728828047841, 0.047287980882073311)
Margin of Error: 23.6600204221%
p-value: 3.86256381291e-05


In [45]:
CI[0]/p_b, p_diff/p_b, CI[1]/p_b

(0.26021509491595141, 0.49681529913654854, 0.73341550335714567)

## 4. Write a story describing the statistical significance in the context or the original problem.

At the start of this analysis, we set out to understand if there was a difference between call-back rates for applicants of different races. To understand this, we looked at call-back data for 4,870 resumes. We split this dataset between resumes with names that sounded black (b) and white (w). After comparing rates between the two groups, we found that we are highly confident that a difference does indeed exist between black and white applicants (as far as we can tell from the names on resumes).

The white applicants had almost 50% better chance of receiving a call back than black applicants. The margin of error was fairly wide, but we are 95% confident that the white applicants call-back odds were at least 26% better and at best 73% better. These results were found to be highly statistically significant with a p-value for the study of 3.86e-5.

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, this study only looked at one variable (race) as a factor for call-back success. We do not know if it is the most important factor. We would need to take a look at other factors, such as previous experience, honors, skills, etc., to determine which factor is the most important in terms of increasing call-back rate.